In [16]:
import numpy as np
import pyswarms as ps

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NeighborhoodComponentsAnalysis

In [20]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO

In [24]:
nca = NCA(optimization_func=PSO, max_iter=1000, k=5)

In [25]:
nca.fit(X_train,y_train)
y_pred = nca.predict(X_test)

In [26]:
np.mean(y_pred == y_test)

0.8833333333333333

In [9]:
def Gradient(X, same_class_mask, transformation,max_iter=1000):
    def alt_obj_func(x, same_class_mask, X_train):
        from sklearn.metrics import pairwise_distances
        from sklearn.utils.extmath import softmax
        
        X_embedded = np.dot(X_train,x.T)
        p_ij = pairwise_distances(X_embedded, squared=True)
        p_ij = softmax(-p_ij)
        np.fill_diagonal(p_ij, 0.0)
        
        pi = same_class_mask * p_ij
        
    
    def obj_func(x, same_class_mask, X_train):
        from sklearn.metrics import pairwise_distances
        from sklearn.utils.extmath import softmax
        
        for i in range(X_train.shape[0]):
            
            X_embedded = np.dot(X_train, x.T)
            p_ij = pairwise_distances(X_embedded, squared=True)
            p_ij = softmax(-p_ij)
            np.fill_diagonal(p_ij, 0.0)
            
            p = 0.0
            for j in range(X_train.shape[0]):
                if same_class_mask[i][j]:
                    p += p_ij[i][j]
            
            primeiro_termo = np.zeros( (x.shape[1],x.shape[1]) )
            segundo_termo = np.zeros( (x.shape[1],x.shape[1]) )
            
            for k in range(X_train.shape[0]):
                if i == k: continue
                
                xik = X_train[i] - X_train[k]
                pik = p_ij[i][k]
                term = pik * np.outer(xik,xik)
                primeiro_termo += term
                if same_class_mask[i][j]:
                    segundo_termo += term
            primeiro_termo *= p
            
            x += 0.5 * (primeiro_termo - segundo_termo)
        return x
    
    transformation = transformation.reshape(-1, X.shape[1])
    for it in range(max_iter):
        transformation = obj_func(transformation, same_class_mask, X)

    return np.ravel(transformation)

In [17]:
X = np.array( [[0,0], [0.,0.1], [0.3,0.1], [0.3, 0.0]] )
y = np.array( [0, 1, 1, 0] )
A = np.eye(X.shape[1])

In [18]:
mask = y[:, np.newaxis] == y[np.newaxis, :]

In [21]:
mask

array([[ True, False, False,  True],
       [False,  True,  True, False],
       [False,  True,  True, False],
       [ True, False, False,  True]])

In [26]:
def alt_obj_func(x, same_class_mask, X_train):
        from sklearn.metrics import pairwise_distances
        from sklearn.utils.extmath import softmax
        
        X_embedded = np.dot(X_train,x.T)
        p_ij = pairwise_distances(X_embedded, squared=True)
        print(f'before softmax\n {p_ij}')
        p_ij = softmax(-p_ij)
        np.fill_diagonal(p_ij, 0.0)
        print(f'after softmax\n {p_ij}')
        pi = same_class_mask * p_ij
        print(pi)

In [27]:
alt_obj_func(A, mask, X)

before softmax
 [[0.   0.01 0.1  0.09]
 [0.01 0.   0.09 0.1 ]
 [0.1  0.09 0.   0.01]
 [0.09 0.1  0.01 0.  ]]
after softmax
 [[0.         0.25993621 0.23756381 0.23995137]
 [0.25993621 0.         0.23995137 0.23756381]
 [0.23756381 0.23995137 0.         0.25993621]
 [0.23995137 0.23756381 0.25993621 0.        ]]
[[0.         0.         0.         0.23995137]
 [0.         0.         0.23995137 0.        ]
 [0.         0.23995137 0.         0.        ]
 [0.23995137 0.         0.         0.        ]]


In [16]:
nca_grad = NCA(optimization_func=Gradient, max_iter=1000, k=5)

In [18]:
nca_grad.fit(X_train, y_train)
y_pred_grad = nca_grad.predict(X_test)
np.mean(y_test == y_pred_grad)

0.9833333333333333